# 사용 패키지

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import argparse
import easydict
from torch import autograd

In [ ]:
from torch.utils import data
from torchvision import datasets, transforms
import os

# 데이터 로드

In [ ]:
total = pd.read_csv('./data/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [ ]:
total.head()

In [ ]:
print(total.shape)

In [ ]:
total.isnull().sum()

# 전처리

In [ ]:
total['구분'].unique()

In [ ]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map) # 이 과정은 A~H 까지의 알파벳을 숫자로 매핑(대치)시키는 과정

In [ ]:
total['구분']

In [ ]:
d_map

In [ ]:
total['연월일'] = pd.to_datetime(total['연월일']) # 시계열 데이터로

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "/USER/d2coding.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(y=total['공급량'], x=total['연월일'])
plt.xlabel('Time')
plt.ylabel('Supply')
plt.show()

In [ ]:
# 이를 통해 매 겨울 시기에 공급량이 많았다는 것을 알 수 있다. 

In [ ]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [ ]:
total

In [ ]:
# 연속형 변수인 공급량 컬럼은 따로 drop 하자
scale_cols = ['공급량']
cont_var = total[scale_cols]
# total.drop(columns=['공급량'], inplace=True)
etc = total.drop(columns=['공급량'])
print(cont_var)
print(etc)

In [ ]:
# 데이터 정규화: continuous variable
from sklearn.preprocessing import MinMaxScaler

cont_var.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()

df_scaled = scaler.fit_transform(cont_var)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

In [ ]:
total = pd.concat([etc,df_scaled], axis=1)
total

In [ ]:
# pytorch lstm model 정의

from torch.autograd import Variable

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        # 상대적인 결과를 보고 감으로. 데이터가 많을 때는 레이어 수를 늘리는 게 보편적. 
        # 분류는 클래스 개수로 해야 나중에 softmax를 취할 수 있을 것 regression 해야 할 feature의 개수가 마지막 인자가 될 것. 
        # self.fc = nn.Sequential(
        #     nn.Linear(hidden_size, 50),
        #     nn.Linear(50, 30),
        #     nn.Linear(30, 10),
        #     # nn.Linear(10,1)
        #     nn.Linear(10, num_classes)
        # )
        self.fc = nn.Linear(hidden_size, num_classes)
        # 마지막 달 값을 평균을 내면 하나의 값이 나올 건데, 평균값으로 뒤의 
        # 맨 마지막 한달치 이런걸로 
    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        # 학습 때 y가 같이 들어간 x일 것
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        return out
    


In [ ]:
def dataset_with_window(x, y, window_size):
    x_list = []
    y_list = []   
    for i in range(len(x)-window_size):
        x_list.append(np.array(x.iloc[i:(i+window_size)]))
        if y != 0:
            y_list.append(np.array(y.iloc[i+window_size]))
    if y != 0:
        return torch.FloatTensor(x_list).to(device), torch.FloatTensor(y_list).to(device).view([-1, 1])
    else:
        return torch.FloatTensor(x_list).to(device)



In [ ]:
features = ['구분','year', 'month', 'day', 'weekday', '시간']
label = ['공급량']
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]
test_size = len(total[total['year'].isin(val_years)])
# 원래 윈도우 사이즈는 천단위, 만단위 는 쉽지 않고 몇백단위까지가 최대로 돌릴 수 있는 것
# window_size = int(test_size/2) # 반년치의 데이터를 window_size로 정의하겠다. 
window_size = 24*30
x = total[features]
y = total[label]
print(x.shape, y.shape)
print(y)
print("window_size:", window_size)
X, Y = dataset_with_window(x, y, window_size)

In [ ]:
print(X)
print(Y)

In [ ]:
# 이제 train 과 val로 나눠줄 차례
# 기준 length는 2018년도 데이터셋의 길이로 함
X_train = X[:test_size]
X_test = X[test_size:]
y_train = Y[:test_size]
y_test = Y[test_size:]


In [ ]:
print(X_train.size())
print(y_train.size())
# 여기서 진짜 잘못했다.. .test_size를 잘못 설정을 했어.

In [ ]:
print(X_test.size())
print(y_test.size())

In [ ]:
# 미니 배치 형식으로 맞춰줘
# loader를 x, y 같이 붙여줄 필요가 없음

# train 용은 x, y 두 개가 같이 들어가는 로더를 만들고
# 테스트 로더를 x만 들어가게 한다음에 학습 돌릴 때 train time일 떄는 loader가 뱉어주는 값이 둘 다 있게 하고,
# 테스트 때는 
train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)

# 미니 배치가 너무 크면 학습이 덜 될수 있다. 
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

In [ ]:
# parameter config 
# 딥러닝 학습할 때는 미니 배치화해서 학습을 해야 한다. 지금은
# 36만개를 통째로 넣고 있어서 문제가 있다. 
num_epochs = 2 # 200
learning_rate = 0.1 #0.01

input_size = 6
hidden_size = 2
num_layers = 1

num_classes = 1

# 학습

In [ ]:
torch.cuda.empty_cache() # PyTorch thing

with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
model = LSTM(num_classes, input_size, hidden_size, num_layers).to(device)

criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
# 일단 지금처럼 LSTM 하나라도 지금처럼 하고,
# LSTM 레이어를 늘려보는 것도 다음 단계로 좋다. 

hist = np.zeros(num_epochs)    
loss_graph = [] # 그래프 그릴 목적인 loss.
n = len(train_loader)

for t in range(num_epochs):   
    
    for data in train_loader:

        seq, target = data # 배치 데이터.
        # Forward pass
        out = model(seq)   # 모델에 넣고,
        loss = criterion(out, target) # output 가지고 loss 구하고,
        hist[t] = loss.item()
        
        optimizer.zero_grad() # 
        loss.backward() # loss가 최소가 되게하는 
        optimizer.step() # 가중치 업데이트 해주고,
    # if t % 10 == 0 and t !=0:
        print('[epoch: %d] MAE: %.4f'%(t, loss.item()))
    # validation은?
    # if t % 10 == 0:
        # with torch.no_grad():
            # 여기서 validation 해야해 

In [ ]:
MODEL_DIR = '/USER/kaggle/2nd_inclass/'
torch.save(model, os.path.join(MODEL_DIR, 'model.pt'))

# 추론 및 결과 제출

In [ ]:
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
submission.shape

In [ ]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [ ]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [ ]:
test['구분'] = test['구분'].map(d_map)

In [ ]:
test_x = test[features]

In [ ]:
test_x

In [ ]:
test_x = dataset_with_window(test_x, 0, window_size)

In [ ]:
test_x.shape

In [ ]:
# 추론
model.eval()
preds = model(test_x)

In [ ]:
print(preds.shape)

In [ ]:
submission['공급량'] = preds

In [ ]:
submission.to_csv('lstm1.csv', index=False)

In [ ]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score